Das ist das fertige Notebook, welches aus den Test zu Stopwords hervorgegangen ist. 
Es sind jetzt erstmal fünf indecies drinnen, Standard (Terrier), Terrier_costum, Terrier_query_occur, ChatGPT und Nostopwords.
Wobei Terrier_costum und Terrier_query_occur natürlich bisschen Overfitting sind, aber es sind auch die besten.

In [1]:
#Standard Imports für alle Notebooks

!pip3 install tira ir-datasets python-terrier nltk scikit-learn spacy

from tira.third_party_integrations import ensure_pyterrier_is_loaded, persist_and_normalize_run
from tira.rest_api_client import Client
import pyterrier as pt
import spacy
import nltk
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

ensure_pyterrier_is_loaded()
tira = Client()

pt_dataset = pt.get_dataset('irds:ir-lab-sose-2024/ir-acl-anthology-20240504-training')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 24.7 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 KB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 775.1/775.1 KB 19.7 MB/s eta 0:00:0000:01


PyTerrier 0.10.0 has loaded Terrier 5.8 (built by craigm on 2023-11-01 18:05) and terrier-helper 0.0.8

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [ ]:
# Laden der NLTK Ressourcen
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

# Laden der SpaCy-Ressourcen
!python -m spacy download en_core_web_sm

# Laden des SpaCy-Modells
nlp = spacy.load("en_core_web_sm")

In [ ]:
# Methode um Beschreibung des POS-Tags zu bekommen für den NLTK Lemmatizer
def get_wordnet_pos_nltk(treebank_tag):
    """Konvertiert POS-Tag in ein Format, das vom WordNet-Lemmatizer unterstützt wird."""
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

# Funktion um den Text zu lemmatizen für NLTK Lemmatizer
def lemmatize_text_nltk(text):
    lemmatizer = WordNetLemmatizer()
    tokens = word_tokenize(text)
    pos_tags = nltk.pos_tag(tokens)
    lemmatized_tokens = [lemmatizer.lemmatize(token, get_wordnet_pos_nltk(tag)) for token, tag in pos_tags]
    return ' '.join(lemmatized_tokens)

# Funktion zur Vorverarbeitung von Dokumenten für den NLTK Lemmatizer
def preprocess_documents_nltk(documents):
    """Anwendet Lemmatization auf alle Dokumente."""
    for doc in documents:
        doc['text'] = lemmatize_text_nltk(doc['text'])
        yield doc

# Definition der Funktion zum Erstellen eines Index für NLTK Lemmatizer
def create_index_nltk(documents, stopwords):
    indexer = pt.IterDictIndexer("/tmp/index", overwrite=True, meta={'docno': 100, 'text': 20480}, stopwords=stopwords)
    index_ref = indexer.index(preprocess_documents_nltk(documents))
    return pt.IndexFactory.of(index_ref)

In [ ]:
# Definition der Funktion zur Lemmatization eines Textes mit SpaCy
def lemmatize_text_spacy(text):
    """Lemmatiziert den gegebenen Text mit SpaCy."""
    doc = nlp(text)
    lemmatized_tokens = [token.lemma_ for token in doc]
    return ' '.join(lemmatized_tokens)

# Definition der Funktion zur Vorverarbeitung von Dokumenten
def preprocess_documents_spacy(documents):
    """Anwendet Lemmatization auf alle Dokumente."""
    for doc in documents:
        doc['text'] = lemmatize_text_spacy(doc['text'])
        yield doc

# Definition der Funktion zum Erstellen eines Index
def create_index_spacy(documents, stopwords):
    indexer = pt.IterDictIndexer("/tmp/index", overwrite=True, meta={'docno': 100, 'text': 20480}, stopwords=stopwords)
    index_ref = indexer.index(preprocess_documents_spacy(documents))
    return pt.IndexFactory.of(index_ref)


In [2]:
#Funktion um eigene Indecies zu erstellen
def create_index(path, documents, stopwords, stemmer):
    indexer = pt.IterDictIndexer(path, overwrite=True, meta={'docno': 100, 'text': 20480}, stopwords=stopwords, stemmer=stemmer)
    index_ref = indexer.index(documents)
    return pt.IndexFactory.of(index_ref)

#Funktion um aus einem txt-file eine Python Liste zu machen
def read_text_file_to_array(file_path):
    try:
        with open(file_path, 'r') as file:
            lines = file.readlines()
            # Remove newline characters and convert to integers
            array = [(line.strip()) for line in lines]
            return array
    except FileNotFoundError:
        print(f"File {file_path} not found.")
        return None

In [ ]:
#Die mittels fine tuning verbesserte Stopwordliste auf Basis von der Pyterrier Standard Stopwordliste
terrier_costum_stopwords = read_text_file_to_array('../terrier-costum.txt')
#Stopwordliste, aus der Pyterrier Stopwordliste, aber nur mit den Worten die in den Querys vorkommen auf die getestet wird
terrier_query_occur_stopwords = read_text_file_to_array('../terrier-query.txt')
#Stopwordliste, welche von ChatGPT kommt
chatgpt_stopwords = read_text_file_to_array('../chatgpt-stopwordlist.txt')


#Standardindex von Tira, welcher die ganzen Standardsettings von Pyterrier benutzt
standard_index = tira.pt.index('ir-lab-sose-2024/tira-ir-starter/Index (tira-ir-starter-pyterrier)', pt_dataset)

#Index der optimierten Pyterrier Stopwordliste
terrier_costum_stopword_index = create_index('/tmp/index/', pt_dataset.get_corpus_iter(), terrier_costum_stopwords, 'porter')

#Index zu der query Vorkommen Stopwordliste
terrier_query_occur_stopword_index = create_index('/tmp/index1/', pt_dataset.get_corpus_iter(), terrier_query_occur_stopwords, 'porter')

#Index zur ChatGPT Stopwordlist
chatgpt_stopword_index = create_index('/tmp/index2/', pt_dataset.get_corpus_iter(), chatgpt_stopwords, 'porter')

#Index ohne Stopwords
nostopwords_index = create_index('/tmp/index3/', pt_dataset.get_corpus_iter(), [], 'porter')


#Die jeweiligen Retrievalmodelle mit Weightingmodell XSqrA_M
XSqrA_M = pt.BatchRetrieve(standard_index, wmodel='XSqrA_M')
XSqrA_M_terrier_costum_stopwords = pt.BatchRetrieve(terrier_costum_stopword_index, wmodel='XSqrA_M')
XSqrA_M_terrier_query_occur_stopwords = pt.BatchRetrieve(terrier_query_occur_stopword_index, wmodel='XSqrA_M')
XSqrA_M_chatgpt_stopwords = pt.BatchRetrieve(chatgpt_stopword_index, wmodel='XSqrA_M')
XSqrA_M_nostopwords = pt.BatchRetrieve(nostopwords_index, wmodel='XSqrA_M')

In [ ]:
#Bei STopwords die Stopwords eingeben die man will
costum_index_nltk =create_index_nltk(pt_dataset.get_corpus_iter(), stopwords)
costum_index_spacy = create_index_spacy(pt_dataset.get_corpus_iter(), stopwords)

# Lade Index von tira
index = tira.pt.index('ir-lab-sose-2024/tira-ir-starter/Index (tira-ir-starter-pyterrier)', pt_dataset)

# Erstellen der BatchRetrieve-Instanzen
XSqrA_M = pt.BatchRetrieve(index, wmodel="XSqrA_M")
XSqrA_M_nltk = pt.BatchRetrieve(costum_index_nltk, wmodel="XSqrA_M")
XSqrA_M_spacy = pt.BatchRetrieve(costum_index_spacy, wmodel="XSqrA_M")

# Experiment wie unten


In [4]:
#Evaluation
pt.Experiment(
    [
        XSqrA_M,
        XSqrA_M_terrier_costum_stopwords,
        XSqrA_M_terrier_query_occur_stopwords,
        XSqrA_M_chatgpt_stopwords,
        XSqrA_M_nostopwords
    ],
    pt_dataset.get_topics(),
    pt_dataset.get_qrels(),
    ["ndcg_cut.10", "recip_rank", "recall_100", "P_10"],
    names = [
        "XSqrA_M",
        "XSqrA_M_terrier_costum_stopwords",
        "XSqrA_M_terrier_query_occur_stopwords",
        "XSqrA_M_chatgpt_stopwords",
        "XSqrA_M_nostopwords"
    ]
)

There are multiple query fields available: ('text', 'title', 'query', 'description', 'narrative'). To use with pyterrier, provide variant or modify dataframe to add query column.


,name,ndcg_cut.10,recip_rank,recall_100,P_10
0,XSqrA_M,0.448211,0.650289,0.592214,0.400000
1,XSqrA_M_terrier_costum_stopwords,0.455100,0.664894,0.593577,0.407353
2,XSqrA_M_terrier_query_occur_stopwords,0.451725,0.661387,0.591281,0.401471
3,XSqrA_M_chatgpt_stopwords,0.433872,0.656496,0.593599,0.379412
4,XSqrA_M_nostopwords,0.424464,0.666510,0.585498,0.375000
